In [ ]:
%load_ext autoreload
%autoreload 2

import time
import numpy as np
from random import choice, seed
import matplotlib.pyplot as plt

from pydrake.geometry.optimization import HPolyhedron, Point, VPolytope

from spp.bezier import BezierSPP

In [ ]:
class Cell:
    """A cell in the maze. A maze "Cell" is a point in the grid which may be surrounded by walls to
    the north, east, south or west.
    """

    # A wall separates a pair of cells in the N-S or W-E directions.
    wall_pairs = {'N': 'S', 'S': 'N', 'E': 'W', 'W': 'E'}

    def __init__(self, x, y):
        """Initialize the cell at (x,y). At first it is surrounded by walls."""

        self.x, self.y = x, y
        self.walls = {'N': True, 'S': True, 'E': True, 'W': True}

    def has_all_walls(self):
        """Does this cell still have all its walls?"""

        return all(self.walls.values())

    def knock_down_wall(self, other, wall):
        """Knock down the wall between cells self and other."""

        self.walls[wall] = False
        other.walls[Cell.wall_pairs[wall]] = False


class Maze:
    """A Maze, represented as a grid of cells."""

    def __init__(self, nx, ny):
        """Initialize the maze grid.
        The maze consists of nx x ny cells.

        """

        self.nx, self.ny = nx, ny
        self.map = [[Cell(x, y) for y in range(ny)] for x in range(nx)]

    def cell_at(self, x, y):
        """Return the Cell object at (x,y)."""

        return self.map[x][y]

    def __str__(self):
        """Return a (crude) string representation of the maze."""

        maze_rows = ['-' * self.nx * 2]
        for y in range(self.ny):
            maze_row = ['|']
            for x in range(self.nx):
                if self.map[x][y].walls['E']:
                    maze_row.append(' |')
                else:
                    maze_row.append('  ')
            maze_rows.append(''.join(maze_row))
            maze_row = ['|']
            for x in range(self.nx):
                if self.map[x][y].walls['S']:
                    maze_row.append('-+')
                else:
                    maze_row.append(' +')
            maze_rows.append(''.join(maze_row))
        return '\n'.join(maze_rows)
    
    def plot(self, linewidth):
        plt.gca().axis('off')
        
        # Pad the maze all around by this amount.
        width = self.nx
        height = self.ny
        
        # Draw the South and West maze borders.
        for x in range(self.nx):
            for y in range(self.ny):
                if self.cell_at(x, y).walls['S'] and (x != 0 or y != 0):
                    plt.plot([x, x + 1], [y, y], c='k', linewidth=linewidth)
                if self.cell_at(x, y).walls['W']:
                    plt.plot([x, x], [y, y + 1], c='k', linewidth=linewidth)
                    
        # Draw the North and East maze border, which won't have been drawn
        # by the procedure above.
        plt.plot([0, width - 1], [height, height], c='k', linewidth=linewidth)
        plt.plot([width, width], [0, height], c='k', linewidth=linewidth)
        
    def find_valid_neighbours(self, cell):
        """Return a list of unvisited neighbours to cell."""

        delta = [('W', (-1, 0)),
                 ('E', (1, 0)),
                 ('S', (0, -1)),
                 ('N', (0, 1))]
        neighbours = []
        for direction, (dx, dy) in delta:
            x2, y2 = cell.x + dx, cell.y + dy
            if (0 <= x2 < self.nx) and (0 <= y2 < self.ny):
                neighbour = self.cell_at(x2, y2)
                if neighbour.has_all_walls():
                    neighbours.append((direction, neighbour))
        return neighbours

    def make_maze(self):
        # Total number of cells.
        n = self.nx * self.ny
        cell_stack = []
        current_cell = self.cell_at(0, 0)
        # Total number of visited cells during maze construction.
        nv = 1

        while nv < n:
            neighbours = self.find_valid_neighbours(current_cell)

            if not neighbours:
                # We've reached a dead end: backtrack.
                current_cell = cell_stack.pop()
                continue

            # Choose a random neighbouring cell and move to it.
            direction, next_cell = choice(neighbours)
            current_cell.knock_down_wall(next_cell, direction)
            cell_stack.append(current_cell)
            current_cell = next_cell
            nv += 1

In [ ]:
seed(2)
nx, ny = 30, 30
start = np.array([0.5, 0.02])
goal = np.array([nx - 0.5, ny-0.02])

maze = Maze(nx, ny)
maze.make_maze()
maze.cell_at(15, 21).knock_down_wall(maze.cell_at(15, 22), "N")

regions = []
edges = []
for x in range(nx):
    for y in range(ny):
        regions.append(HPolyhedron.MakeBox([x, y], [x+1., y+1.]))
        C = y + x * maze.ny
        if not maze.map[x][y].walls['N']:
            edges.append((C, C + 1))
        if not maze.map[x][y].walls['S']:
            edges.append((C, C - 1))
        if not maze.map[x][y].walls['E']:
            edges.append((C, C + maze.ny))
        if not maze.map[x][y].walls['W']:
            edges.append((C, C - maze.ny))

plt.figure(figsize=(20,20))
plt.axis('equal')
maze.plot(4)
plt.plot(start[0], start[1], 'b*', markersize=15)
plt.plot(goal[0], goal[1], 'g*', markersize=15)

In [ ]:
convex_relaxation = True

start_time = time.time()
bspp = BezierSPP(regions, 3, 2, edges)
bspp.addTimeCost(1)
bspp.addPathLengthCost(1)
bspp.addVelocityLimits([-1, -1], [1, 1])
build_time = time.time()
print("Bezier SPP build time:", build_time-start_time)

b_traj, result, hard_result = bspp.SolvePath(start, goal, convex_relaxation, False, [[0], [-1]])
print("Bezier SPP solve time:", time.time()-build_time)

In [ ]:
plt.figure(figsize=(10,10))
plt.axis('equal')
maze.plot(3)
plt.plot(start[0], start[1], 'b*', markersize=15)
plt.plot(goal[0], goal[1], 'g*', markersize=15)

t_steps = np.linspace(b_traj.start_time(), b_traj.end_time(), 5001)
values = np.squeeze([b_traj.value(t) for t in t_steps])
plt.plot(values[:, 0], values[:, 1], "b-", linewidth=3)

alt_path = np.array([[15.0, 21.5], [16.5, 21.5], [16.5, 20.5], [15.5, 20.5], [15.5, 16.5], [16.5, 16.5],
                     [16.5, 14.5], [17.5, 14.5], [17.5, 12.5], [14.5, 12.5], [14.5, 10.5], [16.5, 10.5],
                     [16.5, 11.5], [25.5, 11.5], [25.5, 13.5], [26.5, 13.5], [26.5, 10.5], [24.5, 10.5],
                     [24.5,  9.5], [27.5,  9.5], [27.5,  6.5], [25.5,  6.5], [25.5,  7.5], [26.5,  7.5],
                     [26.5,  8.5], [24.5,  8.5], [24.5,  5.5], [26.5,  5.5], [26.5,  4.5], [24.5,  4.5],
                     [24.5,  3.5], [23.5,  3.5], [23.5,  4.5], [22.5,  4.5], [22.5,  3.5], [21.5,  3.5],
                     [21.5,  1.5], [22.5,  1.5], [22.5,  0.5], [24.5,  0.5], [24.5,  1.5], [23.5,  1.5],
                     [23.5,  2.5], [25.5,  2.5], [25.5,  1.5], [26.5,  1.5], [26.5,  3.5], [27.5,  3.5],
                     [27.5,  2.5], [28.5,  2.5], [28.5,  1.5], [29.5,  1.5], [29.5, 11.5], [28.5, 11.5],
                     [28.5, 12.5], [27.5, 12.5], [27.5, 13.5], [29.5, 13.5], [29.5, 14.5], [28.5, 14.5],
                     [28.5, 18.5], [27.5, 18.5], [27.5, 17.5], [25.5, 17.5], [25.5, 19.5], [26.5, 19.5],
                     [26.5, 22.5], [25.5, 22.5], [25.5, 21.5], [23.5, 21.5], [23.5, 20.5], [22.5, 20.5],
                     [22.5, 16.5], [21.5, 16.5], [21.5, 22.5], [24.5, 22.5], [24.5, 23.5], [23.5, 23.5],
                     [23.5, 24.5], [27.5, 24.5], [27.5, 23.5], [28.5, 23.5], [28.5, 21.5], [29.5, 21.5],
                     [29.5, 24.5], [28.5, 24.5], [28.5, 25.5], [29.5, 25.5], [29.5, 30]])
plt.plot(alt_path[:, 0], alt_path[:, 1], "k:", linewidth=2)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))

t_steps = np.linspace(b_traj.start_time(), b_traj.end_time(), 5001)
values = np.squeeze([b_traj.value(t) for t in t_steps])
velocity_traj = np.squeeze([b_traj.EvalDerivative(t) for t in t_steps])

ax.plot(t_steps, velocity_traj[:, 0], label="x velocity")
ax.plot(t_steps, velocity_traj[:, 1], label="y velocity")
ax.plot([t_steps[0], t_steps[-1]], [1, 1], 'k--', label="limits")
ax.plot([t_steps[0], t_steps[-1]], [-1, -1], 'k--')
ax.legend(loc="right")
ax.set_xlabel("Time")
ax.set_ylabel("Velocity")